In [1]:
from utils import login, get_atm_strike, get_option_symbols, get_quotes
from config import Config
from logger_setup import app_logger
config = Config('creds/config.yaml', 'creds/tbs_rules.yaml')
api = login(config)
async def _get_atm_strike():
    try:
        atm_strike = await get_atm_strike(
            config.get_rule('tsymbol'), 
            lambda e, t: api.get_quotes(e, t)
        )
        if not atm_strike:
            app_logger.error("Failed to get ATM strike. Aborting simulation.")
            return None
        return atm_strike
    except Exception as e:
        app_logger.error(f"Error getting ATM strike: {e}")
        return None

async def _get_option_symbols(atm_strike):
    try:
        strikes = {
            'sce': (atm_strike + config.get_rule('sotm_points') + config.get_rule('bias_points'), 'CE'),
            'spe': (atm_strike - config.get_rule('sotm_points') + config.get_rule('bias_points'), 'PE'),
        }
        option_symbols = await get_option_symbols(config.get_rule('tsymbol'), strikes)
        if not option_symbols:
            app_logger.error("Failed to get option symbols. Aborting simulation.")
            return None
        return option_symbols
    except Exception as e:
        app_logger.error(f"Error getting option symbols: {e}")
        return None

In [2]:
atm_strike = await _get_atm_strike()
option_symbols = await _get_option_symbols(atm_strike)

In [3]:
from margin_calculator import MarginCalculator
required_margin, final_trade_margin = await margin_calculator.calculate_margin(option_symbols, final_quantity)

{'sce': {'Exchange': 'NFO',
  'Token': 59154,
  'LotSize': 25,
  'Symbol': 'FINNIFTY',
  'TradingSymbol': 'FINNIFTY20AUG24C23200',
  'Expiry': Timestamp('2024-08-20 00:00:00'),
  'Instrument': 'OPTIDX',
  'OptionType': 'CE',
  'StrikePrice': 23200.0},
 'spe': {'Exchange': 'NFO',
  'Token': 59155,
  'LotSize': 25,
  'Symbol': 'FINNIFTY',
  'TradingSymbol': 'FINNIFTY20AUG24P23200',
  'Expiry': Timestamp('2024-08-20 00:00:00'),
  'Instrument': 'OPTIDX',
  'OptionType': 'PE',
  'StrikePrice': 23200.0}}

In [2]:
import pyotp
factor2 = pyotp.TOTP('7E5YER64Y5VE257GY7D265X6472CLICK').now()
print(factor2)



299295


In [ ]:
fno